In [6]:
from pathlib import Path, WindowsPath, PosixPath
# SETTINGS ------------------------------------------------------------------------
CONFIG = {
    "ROOT":             Path('../../..').resolve(),
    "INPUT_LOCATION":   Path("TripSequences", "trips_processed_pinn_3"), 
    "OUTPUT_LOCATION":  Path("TripSequences", "trips_processed_pinn_4"),
}
for key in CONFIG: globals()[key] = CONFIG[key];
print(ROOT)

/home/sieglew/MA-eR-PINN


In [7]:
# LOCATE REPOSITORY/DATASTORAGE IN CURRENT SYSTEM ENVIRONMENT  ---------------------------
# ----------------------------------------------------------------------------------------
import sys, os
if 'ROOT' not in globals(): global ROOT; ROOT = Path('../..').resolve()
print(f"{'-'*60}\nDirectories:\n  {ROOT}:\t\t\t{', '.join([_.name for _ in ROOT.glob('*/')])}")
sys.path.append(os.path.abspath(ROOT))
from data import get_data_path  # paths set in "data/__init__.py"
global DATA_PATH
DATA_PATH = get_data_path()
print(f"  {DATA_PATH}:\t\t\t{', '.join([_.name for _ in DATA_PATH.glob('*/')])}")

# INTERNAL MODULE IMPORTS ----------------------------------------------------------------
# ----------------------------------------------------------------------------------------
from src.utils.data_utils import *
from src.utils.Trainers import *
from src.PINN.physics_model.VehModel import CreateVehicle
from src.PINN.physics_model.Mot_Pwr import estimate_mot_pwr

# NOTEBOOK / SCRIPT SETTINGS -------------------------------------------------------------
# ----------------------------------------------------------------------------------------
global IS_NOTEBOOK
IS_NOTEBOOK = False
try:    # if running in IPython
    shell = get_ipython().__class__.__name__ # type: ignore 
    from IPython.display import display, HTML, Javascript, clear_output
    from IPython.core.magic import register_cell_magic
    @register_cell_magic    # cells can be skipped by using '%%skip' in the first line
    def skip(line, cell): return
    from tqdm.notebook import tqdm as tqdm_nb
    IS_NOTEBOOK = True
    print(f"{'-'*60}\nRunning in notebook mode")
except (NameError, ImportError):    # if running in script
    from tqdm import tqdm as tqdm
    from tabulate import tabulate
    print(f"{'-'*60}\nRunning in script mode")
    
# GENERAL MODULE IMPORTS -----------------------------------------------------------------
# ----------------------------------------------------------------------------------------
import math, time, random, pickle, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import pyarrow.parquet as pq
from copy import deepcopy
from datetime import datetime

from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_squared_error, r2_score
from scipy.signal import savgol_filter

# Suppress all DeprecationWarnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

------------------------------------------------------------
Directories:
  /home/sieglew/MA-eR-PINN:			test, .git, archive, project, data, src
  /mnt/nvme/datasets/sieglew:			TripSequences
------------------------------------------------------------
Running in notebook mode


In [8]:
input_folder = Path(DATA_PATH, INPUT_LOCATION) # Trip parquet files
output_folder = Path(DATA_PATH, OUTPUT_LOCATION) # Trip parquet files
pickle_folder = Path(DATA_PATH, "TripSequences", "trips_processed_pickles") # Processed trip files

all_files = [Path(input_folder, f).name for f in os.listdir(input_folder) if f.endswith(".parquet")]
print(f"{'-'*60}\nTotal Files:\t{len(all_files)}")

------------------------------------------------------------
Total Files:	8612


In [9]:
samples = random.sample(all_files,20)

In [ ]:
collect = []
for n, f in enumerate(all_files):
    #print(n)
    file_code = f.strip(".parquet")
    v_id = file_code.split("_")[0].strip("V")

    if v_id in ["1", "15", "19"]: continue

    # Load parquet
    df = pd.read_parquet(Path(input_folder,f), engine='pyarrow')

    # signals:
    time = df.index.to_numpy()
    emot_pwr_true = df["emot_pwr_cval"].to_numpy()
    en = df["hv_batmomavldischrgen_cval_1"].to_numpy()
    soc = df["hv_bat_soc_cval_bms1"].to_numpy()
    dist = df["hirestotalvehdist_cval_icuc"].to_numpy()
    speed = (df["vehspd_cval_cpc"]/3.6).to_numpy()

    accel = np.diff(speed, prepend=speed[0])
    alt = df["altitude_cval_ippc"].to_numpy()
    road_grad = df["roadgrad_cval_pt"].to_numpy()
    amb_temp = df["airtempoutsd_cval_cpc"].to_numpy()
    vehweight = (df['vehweight_cval_pt']*1000).to_numpy()


    # Load pickle
    with open(Path(pickle_folder, f'{file_code}.pickle'), 'rb') as handle:
        T,C,V_prm = pickle.load(handle) 

    # create emot estimate
    emot_pwr_pred = estimate_mot_pwr(speed, accel, alt, road_grad, amb_temp, vehweight, 
                V_prm["c_w_a"], 
                V_prm["tire_roll_res_coeff"], 
                rot_inertia = 0, 
                eta_mech = .97,
                eta_mot = .95
                )
    df["emot_pwr_pred"] = emot_pwr_pred

    assert len(df) == len(emot_pwr_pred)

    # transform motor power prediction to energy consumption:
    C_nom = 350
    emot_en = np.cumsum(-emot_pwr_pred)/3600 # [kWh]
    emot_soc = emot_en/C_nom*100 + soc[0]
    

    df["start_soc"] = np.full_like(soc, soc[0])
    df["start_en"] = np.full_like(en, en[0])
    # calculate new target
    soc_gradient = np.gradient(soc.flatten(), time)
    df["soc_gradient"] = soc_gradient

    en_gradient = np.gradient(en.flatten(), time)
    df["en_gradient"] = en_gradient
    
    df["emot_soc_pred"] = emot_soc
    df["emot_en_pred"] = emot_en + en[0]

    soc_range = max(soc) - min(soc)
    if soc_range > 5 and len(df) > 599:
        if (emot_soc - soc).mean() <-1 or speed.mean() < 3: 
            collect.append(f)
        else:
            df.to_parquet(Path(output_folder, f), engine='pyarrow')
        '''
        plt.figure(figsize=(12,2))
        plt.xlabel('Time in s')
        plt.ylabel('SOC in %')
        plt.title('Battery State of Charge: Prediction vs. Actual Data')
        plt.plot(soc, label='Actual Data')  # actual plot
        plt.plot(np.arange(0, len(emot_soc), 1), emot_soc, label='Predicted Data')  # predicted plot
        plt.legend()
        '''



    # save:
    #df.to_parquet(Path(output_folder, f), engine='pyarrow')





V18_T374.parquet
V18_T381.parquet
V18_T363.parquet
V18_T147.parquet
V18_T1176.parquet
V18_T846.parquet
V18_T1122.parquet
V18_T409.parquet
V18_T1592.parquet
V18_T576.parquet
V18_T1486.parquet
V18_T1038.parquet
V18_T335.parquet
V18_T230.parquet
V18_T334.parquet
V18_T324.parquet
V18_T116.parquet
V18_T567.parquet
V18_T1340.parquet
V18_T783.parquet
V18_T1285.parquet
V18_T830.parquet
V18_T355.parquet
V11_T66.parquet
V11_T23.parquet
V18_T779.parquet
V18_T1507.parquet
V18_T1139.parquet
V18_T1576.parquet
V18_T578.parquet
V18_T875.parquet
V18_T1056.parquet
V18_T959.parquet
V18_T1234.parquet
V18_T944.parquet
V18_T1323.parquet
V18_T1159.parquet
V18_T967.parquet
V18_T1192.parquet
V18_T907.parquet
V18_T177.parquet
V18_T1298.parquet
V18_T771.parquet
V18_T872.parquet
V17_T4001.parquet
V18_T908.parquet
V18_T309.parquet
V18_T599.parquet
V18_T118.parquet
V13_T119.parquet
V18_T122.parquet
V18_T1191.parquet
V18_T470.parquet
V18_T1417.parquet
V18_T1573.parquet
V18_T1328.parquet
V18_T357.parquet
V18_T1474.pa

In [11]:
len(collect)

0

In [13]:
df.head()

,signal_time,hirestotalvehdist_cval_icuc,altitude_cval_ippc,latitude_cval_ippc,longitude_cval_ippc,accelpdlposn_cval,actdrvtrnpwrprc_cval,actualdcvoltage_pti1,actualspeed_pti1,actualtorque_pti1,...,vehweight_cval_pt,hv_batmomavldischrgen_cval_1,hv_bat_soc_cval_bms1,start_soc,emot_pwr_pred,emot_soc_pred,soc_gradient,en_gradient,start_en,emot_en_pred
0,2022-04-07 04:52:30,0.0000,107.0000,49.067613,8.289306,40.64,19.18,442.645,196.30,16.640570,...,18.0,313.95,94.211875,94.211875,0.000000,94.211875,-0.000938,-0.020,313.95,313.950000
1,2022-04-07 04:52:31,0.0000,107.0000,49.067613,8.289306,22.72,11.37,442.565,541.55,9.734799,...,18.0,313.93,94.210938,94.211875,2.040174,94.211713,-0.000781,-0.025,313.95,313.949433
2,2022-04-07 04:52:32,0.0000,107.0000,49.067612,8.289304,17.92,8.89,442.500,773.05,7.673302,...,18.0,313.90,94.210313,94.211875,7.594988,94.211110,-0.001094,-0.015,313.95,313.947324
3,2022-04-07 04:52:33,0.0000,107.0000,49.067611,8.289296,15.92,7.89,442.450,942.25,6.828195,...,18.0,313.90,94.208750,94.211875,5.896259,94.210642,-0.001250,0.000,313.95,313.945686
4,2022-04-07 04:52:34,0.0045,106.9625,49.067611,8.289281,13.60,7.13,442.390,1116.10,6.130157,...,18.0,313.90,94.207813,94.211875,6.254484,94.210146,-0.001406,0.000,313.95,313.943948
